<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Enforcing inequalities on input conservatives

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:

* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).
* **(Recommended)** [Etienne, Liu, Paschalidis, and Shapiro. General relativistic simulations of black hole-neutron star mergers: Effects of magnetic fields (2012)](https://arxiv.org/abs/1112.0568v4).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1.[**The `apply_inequality_fixes` function**](#apply_inequality_fixes)
    1. [*Function Arguments*](#fargs)
        1. [Inputs](#inputs)
        1. [Outputs](#outputs)
    1. [*Computing Magnetic Quantities*](#barbi_barb_i_barb2_and_barb): *$\bar{B}^{i}$, $\bar{B}_{i}$, $\bar{B}^{2}$, and $\bar{B}$*
    1. [*Computing Momentum Quantities*](#barbdots_hatbarbdots_and_sdots): *$\bar{B}\cdot\tilde{S}$, $\hat{\bar{B}}\cdot\tilde{S}$, and $\tilde{S}^{2}$*
    1. [*Modifying $\tilde{\tau}$*](#modifying_tau)
    1. [*Modifying $\tilde{S}_{i}$*](#modifying_tilde_s_i)
        1. [Small $\bar{B}$](#small_b)
        1. [Large $\psi^6$](#large_psi)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

For a given set of primitive variables $\left\{\rho_{b},P,v^i,B^i\right\}$ in the physical range (i.e. $\rho_{b}\geq0$, $P\geq0$ and $\epsilon\geq0$), the corresponding conservative variables $\left\{\rho_{\star},\tilde{\tau},\tilde{S}_{i},\tilde{B}^{i}\right\}$ must satisfy certain inequalities (see appendix A of [Etienne *et al.* (2012)](https://arxiv.org/abs/1112.0568v4) for the full discussion). Here we provide a practical recipe to impose these inequalities approximately to reduce inversion failures, which occur mainly in regions with very low density in the artificial “atmosphere” or inside the BH horizon where high accuracy is difficult to maintain but not crucial. This function is designed for use with the hybrid equation of state, but it does not directly rely on any hybrid-specific features.

<a id='apply_inequality_fixes'></a>

# The `apply_inequality_fixes` function \[Back to [top](#toc)\]
$$\label{apply_inequality_fixes}$$

<a id='fargs'></a>

## Function Arguments \[Back to [top](#toc)\]
$$\label{fargs}$$

First, let's look at the arguments of `apply_inequality_fixes`:
```
void apply_inequality_fixes(
      const GRHayL_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      const primitive_quantities *restrict prims,
      conservative_quantities *restrict cons,
      con2prim_diagnostics *restrict diagnostics)
```

<a id='inputs'></a>

### Inputs
$$\label{inputs}$$

The parameter used from `params` is `psi6threshold`, which is used to trigger fixes.

The EOS quantities used from `eos` are `tau_atm` and `press_atm`, which are used to trigger fixes and reset $\tilde{tau}$.

The metric quantities used from `metric` are the ADM metric (`adm_gxx`, `adm_gxy`, etc.), inverse metric (`adm_gupxx`, `adm_gupxy`, etc.), and conformal factor quantity `psi6` ($\psi^6$). Since auxiliary quantities are used, we recommend use either `GRHayL_enforce_detgtij_and_initialize_metric` or `initialize_metric` to automatically compute these quantities from the ADM quantities.

The primitive quantities used from `prims` are the magnetic quantities (`Bx`, `By`, `Bz`).

The conservative quantities used from `cons` are the densitized density $\tilde{\rho}$ (`rho`), densitized energy variable `tau`, and the densitized momentum (`S_x`, `S_y`, `S_z`).

<a id='outputs'></a>

### Outputs
$$\label{outputs}$$

The outputs are also in the `cons` struct. Depending on the inequalities violated, `tau` and the densitized momentum may be changed.

Additionally, `diagnostics->failure_checker` stores information about which inequalities were violated. Each digit of `failure_checker` represents a different condition in the Con2Prim gem, so the individual failures are simply added to the variable. If `tau` is reset, then $10^6$ is added to `failure_checker`. If the momentum is rescaled, then a factor is added to `failure_checker` depending on the failure condition. The first condition adds $10^7$, and the second condition adds $10^8$.

<a id='barbi_barb_i_barb2_and_barb'></a>

## Computing Magnetic Quantities: $\bar{B}^{i}$, $\bar{B}_{i}$, $\bar{B}^{2}$, and $\bar{B}$ \[Back to [top](#toc)\]
$$\label{barbi_barb_i_barb2_and_barb}$$

We first set

$$
\boxed{\bar{B}^{i} = \frac{B^{i}}{\sqrt{4\pi}}}\ ,
$$

and

$$
\bar{B}_{i} = \gamma_{ij}\bar{B}^{j} \implies 
\boxed{
\left\{
\begin{matrix}
\bar{B}_{x} = \gamma_{xx}\bar{B}^{x} + \gamma_{xy}\bar{B}^{y} + \gamma_{xz}\bar{B}^{z}\\
\bar{B}_{y} = \gamma_{yx}\bar{B}^{x} + \gamma_{yy}\bar{B}^{y} + \gamma_{yz}\bar{B}^{z}\\
\bar{B}_{z} = \gamma_{zx}\bar{B}^{x} + \gamma_{zy}\bar{B}^{y} + \gamma_{zz}\bar{B}^{z}
\end{matrix}
\right.
}\ ,
$$

then

$$
\bar{B}^{2} \equiv B_{i}B^{i} \implies \boxed{\bar{B}^{2} = B_{x}B^{x} + B_{y}B^{y} + B_{z}B^{z}}\ ,
$$

and finally

$$
\boxed{\bar{B} \equiv \sqrt{\bar{B}^{2}}}\ .
$$

The next part of the code is written to prevent [floating-point underflow](https://en.wikipedia.org/wiki/Arithmetic_underflow). We compute $\bar{B}$ in a different way. We start by evaluating

$$
\bar{B}_\mathrm{check} = \left|\bar{B}^{x}\right| + \left|\bar{B}^{y}\right| + \left|\bar{B}^{z}\right|\ ,
$$

and verifying whether that is a very small, positive number. Then, we determine the largest component of $\bar{B}_\mathrm{check}$: 

$$
\bar{B}_{\max} = \max\left(\left|\bar{B}^{x}\right|,\left|\bar{B}^{y}\right|,\left|\bar{B}^{z}\right|\right)\ .
$$

Then, we rescale $\bar{B}_{i}$ and $\bar{B}^{i}$ using

$$
\left(\bar{B}^{i}\right)_\mathrm{tmp} \equiv \frac{\bar{B}^{i}}{\bar{B}_{\max}}\ ,\quad
\left(\bar{B}_{i}\right)_\mathrm{tmp} \equiv \frac{\bar{B}_{i}}{\bar{B}_{\max}}\ ,
$$

and finally recompute $\bar{B}$

$$
\bar{B} = \left[\left(\bar{B}_{i}\right)_\mathrm{tmp}\left(\bar{B}^{i}\right)_\mathrm{tmp}\right]\bar{B}_{\max}\ .
$$

<a id='barbdots_hatbarbdots_and_sdots'></a>

## Computing Momentum Quantities: $\bar{B}\cdot\tilde{S}$, $\hat{\bar{B}}\cdot\tilde{S}$, and $\tilde{S}^{2}$ \[Back to [top](#toc)\]
$$\label{barbdots_hatbarbdots_and_sdots}$$

Then we compute

$$
\bar{B} \cdot \tilde{S} = \bar{B}_{i}\tilde{S}^{i} = \bar{B}_{x}\tilde{S}^{x} + \bar{B}_{y}\tilde{S}^{y} + \bar{B}_{z}\tilde{S}^{z}\ .
$$

and

$$
\hat{\bar{B}}\cdot\tilde{S} \equiv \frac{\bar{B} \cdot \tilde{S}}{\bar{B}}\ .
$$

However, if $\bar{B} \ll 1$, we set $\hat{\bar{B}}\cdot\tilde{S}=0$. Next we compute

$$
\tilde{S}^{2} \equiv \tilde{S} \cdot \tilde{S} = \gamma^{ij}\tilde{S}_{i}\tilde{S}_{j}\ ,
$$

i.e.

$$
\boxed{
\begin{align}
\tilde{S}^{2} &= \gamma^{xx}\left(\tilde{S}_{x}\right)^{2}
                           + \gamma^{yy}\left(\tilde{S}_{y}\right)^{2}
                           + \gamma^{zz}\left(\tilde{S}_{z}\right)^{2}\\
                          &+2\left(
                             \gamma^{xy}\tilde{S}_{x}\tilde{S}_{y}
                            +\gamma^{xz}\tilde{S}_{x}\tilde{S}_{z}
                            +\gamma^{yz}\tilde{S}_{y}\tilde{S}_{z}
                           \right)
\end{align}
}\ .
$$

<a id='modifying_tau'></a>

## Modifying $\tilde{\tau}$ \[Back to [top](#toc)\]
$$\label{modifying_tau}$$

Now, we apply the fix to $\tilde{\tau}$ with the conditional

$$
\tilde{\tau} < \frac{1}{2}\psi^6 \bar{B}^2
$$

If this condition is met, then we reset $\tilde{\tau}$ to

$$
\tilde{\tau} = \tilde{\tau}_\mathrm{atm} \frac{1}{2}\psi^6 \bar{B}^2
$$

<a id='modifying_tilde_s_i'></a>

## Modifying $\tilde{S}_{i}$ \[Back to [top](#toc)\]
$$\label{modifying_tilde_s_i}$$

Following the $\tilde{\tau}$ fix, we prepare to calculate the $S_i$ fix. First, we first compute the quantities from Eqs. (A52), (A53), and (A54) in appendix A of [Etienne *et al.* (2012)](https://arxiv.org/abs/1112.0568v4)

$$
\begin{align}
W_{m} &= \psi^{-6}\left[\left(\hat{\bar{B}}\cdot\tilde{S}\right)^{2}+\tilde{\rho}^{2}\right]^{1/2}\ ,\\
\tilde{S}_{m}^{2} &= \frac{W_{m}^{2}\tilde{S}^{2} + \left(\bar{B}\cdot\tilde{S}\right)^{2}\left(\bar{B}^{2}+2W_{m}\right)}{\left(W_{m}+\bar{B}^{2}\right)^{2}}\ ,\\
W_{\min} &= \psi^{-6}\left(S_{m}^{2}+\tilde{\rho}^{2}\right)^{1/2}\ ,\\
\end{align}
$$

respectively (notice the slightly different notation between the equations above and the one used in the paper). We then calculate

$$
\tilde{\tau}_\mathrm{fluid\ min} = \tilde{\tau} - \frac{1}{2}\psi^{6}\bar{B}^{2} - \frac{\bar{B}^{2}\tilde{S}^{2} - \left(\bar{B}\cdot\tilde{S}\right)^{2}}{2\psi^{6}\left(W_{\min}+\bar{B}^{2}\right)^{2}}
$$

The logic is then split into two branches: small $\bar{B}$ and large $\psi^6$. For both branches, $S_i$ is rescaled by a factor which differs only by whether it uses $\tilde{\tau}$ or $\tilde{\tau}_\mathrm{fluid\ min}$.

<a id='small_b'></a>

### Small $\bar{B}$
$$\label{small_b}$$

This branch is triggered by the condition

$$
\left(\Sigma_i\left|\bar{B}^i\right|\right)^2 < 10^{-32}P_\mathrm{atm}
$$

This is immediately followed by the conditional

$$
\tilde{S}^{2} > 0.999999\ \left(\tilde{\tau}^2+2\tilde{\tau}\tilde{\rho}\right)
$$

which triggers the rescaling

$$
S_i = \sqrt{\frac{0.999999\ \left(\tilde{\tau}^2+2\tilde{\tau}\tilde{\rho}\right)}{\tilde{S}^{2}}} S_i
$$

<a id='large_psi'></a>

### Large $\psi^6$
$$\label{large_psi}$$

This branch is triggered by the condition

$$
\psi^6 > \mathrm{params\rightarrow psi6threshold}
$$

and only if the previous branch does not trigger (`else if`). Additionally, if

$$
\tilde{\tau}_\mathrm{fluid\ min} < 1.001\ \tilde{\tau}_\mathrm{atm}
$$

then we change $\tilde{\tau}_\mathrm{fluid\ min}$ and $\tilde{\tau}$:

$$
\tilde{\tau}_\mathrm{fluid\ min} = 1.001\ \tilde{\tau}_\mathrm{atm}
$$

$$
\tilde{\tau} = \tilde{\tau}_\mathrm{fluid\ min} + \frac{1}{2}\psi^{6}\bar{B}^{2} + \frac{\bar{B}^{2}\tilde{S}^{2} - \left(\bar{B}\cdot\tilde{S}\right)^{2}}{2\psi^{6}\left(W_{\min}+\bar{B}^{2}\right)^{2}}
$$

Finally, rescaling of $S_i$ is triggered by the conditional

$$
\tilde{S}^{2} > 0.999999\ \left(\tilde{\tau}_\mathrm{fluid\ min}^2+2\tilde{\tau}_\mathrm{fluid\ min}\tilde{\rho}\right)
$$

with the rescaling being


$$
S_i = \sqrt{\frac{0.999999\ \left(\tilde{\tau}_\mathrm{fluid\ min}^2+2\tilde{\tau}_\mathrm{fluid\ min}\tilde{\rho}\right)}{\tilde{S}^{2}}} S_i
$$